In [1]:
# Esse notebooks foi gerado com a finalidade de testar o impacto do escalonamento dos dados no desempenho 
# dos modelos de machine learning
# e não deve ser utilizado como um modelo final para produção.
# O objetivo é demonstrar como o escalonamento pode afetar a performance dos modelos de machine learning, 
# especialmente em relação ao RMSE e R2.
# --- IGNORE ---

In [2]:
# =============================================================================
# 1. SETUP E IMPORTAÇÃO DAS BIBLIOTECAS
# =============================================================================
import pandas as pd
import numpy as np
import warnings
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import cross_validate, GroupKFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error

# Ignorar avisos para uma saída mais limpa
warnings.filterwarnings('ignore')

In [5]:
# =============================================================================
# 2. CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================
def carregar_dados():
    """
    Função para carregar os dados.
    """
    try:
        # --- SUBSTITUA O CAMINHO ABAIXO PELO CAMINHO DO SEU ARQUIVO ---
        caminho_do_arquivo = '/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed/dados-organizados-1-1km.csv'
        df = pd.read_csv(caminho_do_arquivo)
        print(f"Dados reais carregados de '{caminho_do_arquivo}'. Formato: {df.shape}")
        return df
    except FileNotFoundError:
        print("AVISO: Arquivo de dados real não encontrado. Usando dados simulados para demonstração.")
        data = {
            'km': 0,
            'pele': 0,
            'velocidade': 0,
            'umidade_relativa': 0,
            'ambiente_umida': 0,
            'trial_id': 0
        }
        return pd.DataFrame(data)

df_exercicio = carregar_dados()

TARGET = 'interna'
FEATURE_SETS = {
    'simple': ['km', 'ibutg', 'pele', 'fc'],
    'simple_com_velocidade': ['km', 'ibutg', 'pele', 'fc', 'velocidade'],
    'ambiental_completo': ['km', 'ibutg', 'umidade_absoluta', 'umidade_relativa',
                           'ambiente_seca', 'ambiente_umida', 'pele', 'fc']
}

X = df_exercicio.drop(columns=[TARGET, 'trial_id'])
y = df_exercicio[TARGET]
groups = df_exercicio['trial_id']
cv_splitter = GroupKFold(n_splits=5)

Dados reais carregados de '/home/usuario-leticia/Desktop/Samuel/leticiaag/tcore/data/processed/dados-organizados-1-1km.csv'. Formato: (750, 18)


In [7]:
# =============================================================================
# 3. HIPERPARÂMETROS OTIMIZADOS (DA SUA TABELA)
# =============================================================================
# Dicionário com os melhores parâmetros para cada combinação de modelo e feature set
OPTIMIZED_PARAMS = {
    'Lasso': {
        'simple': {'alpha': 0.02868},
        'simple_com_velocidade': {'alpha': 0.02394},
        'ambiental_completo': {'alpha': 0.02631}
    },
    'DecisionTree': {
        'simple': {'max_depth': 10, 'min_samples_leaf': 25, 'min_samples_split': 5},
        'simple_com_velocidade': {'max_depth': 10, 'min_samples_leaf': 25, 'min_samples_split': 5},
        'ambiental_completo': {'max_depth': 10, 'min_samples_leaf': 25, 'min_samples_split': 5}
    },
    'RandomForest': {
        'simple': {'max_features': 0.8, 'max_samples': 0.8, 'min_samples_leaf': 10, 'min_samples_split': 30, 'n_estimators': 200},
        'simple_com_velocidade': {'max_features': 0.7, 'max_samples': 0.8, 'min_samples_leaf': 10, 'min_samples_split': 30, 'n_estimators': 200},
        'ambiental_completo': {'max_features': 0.7, 'max_samples': 0.8, 'min_samples_leaf': 10, 'min_samples_split': 30, 'n_estimators': 400}
    },
    'XGBoost': {
        'simple': {'colsample_bytree': 0.5, 'gamma': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 700, 'reg_alpha': 0.1, 'reg_lambda': 6, 'subsample': 0.5},
        'simple_com_velocidade': {'colsample_bytree': 0.5, 'gamma': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 500, 'reg_alpha': 0.1, 'reg_lambda': 8, 'subsample': 0.5},
        'ambiental_completo': {'colsample_bytree': 0.5, 'gamma': 1.0, 'learning_rate': 0.02, 'max_depth': 3, 'n_estimators': 700, 'reg_alpha': 0.1, 'reg_lambda': 8, 'subsample': 0.5}
    }
}

In [8]:
# =============================================================================
# 4. CONFIGURAÇÃO E EXECUÇÃO DO EXPERIMENTO DE AVALIAÇÃO
# =============================================================================
resultados_finais = []
scoring = {
    'r2': 'r2',
    'rmse': make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)
}

print("\n--- Iniciando a avaliação dos modelos com hiperparâmetros otimizados ---")

for feature_set_name, feature_list in FEATURE_SETS.items():
    print(f"\n>>> Avaliando com o conjunto de features: '{feature_set_name}'")
    X_subset = X[feature_list]

    # --- Avaliação do Lasso (Sempre Escalonado) ---
    print("    - Avaliando: Lasso_Scaled")
    lasso_params = OPTIMIZED_PARAMS['Lasso'][feature_set_name]
    pipeline_lasso = Pipeline([('scaler', StandardScaler()), ('model', Lasso(random_state=42, max_iter=20000, **lasso_params))])
    scores = cross_validate(pipeline_lasso, X_subset, y, cv=cv_splitter, scoring=scoring, groups=groups, return_train_score=True)
    resultados_finais.append({
        'experimento': f"Lasso_Scaled ({feature_set_name})",
        'R2_treino': np.mean(scores['train_r2']), 'R2_teste': np.mean(scores['test_r2']),
        'RMSE_treino': -np.mean(scores['train_rmse']), 'RMSE_teste': -np.mean(scores['test_rmse'])
    })

    # --- Avaliação dos Modelos de Árvore (Escalonado vs. Não Escalonado) ---
    for model_name in ['DecisionTree', 'RandomForest', 'XGBoost']:
        params = OPTIMIZED_PARAMS[model_name][feature_set_name]
        
        # Instancia o modelo correto para a iteração atual do loop
        model_instance = None
        if model_name == 'DecisionTree':
            model_instance = DecisionTreeRegressor(random_state=42, **params)
        elif model_name == 'RandomForest':
            model_instance = RandomForestRegressor(random_state=42, **params)
        elif model_name == 'XGBoost':
            model_instance = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **params)

        if model_instance is not None:
            # Cenário NÃO ESCALONADO
            print(f"    - Avaliando: {model_name}_Unscaled")
            pipeline_unscaled = Pipeline([('model', model_instance)])
            scores_unscaled = cross_validate(pipeline_unscaled, X_subset, y, cv=cv_splitter, scoring=scoring, groups=groups, return_train_score=True)
            resultados_finais.append({
                'experimento': f"{model_name}_Unscaled ({feature_set_name})",
                'R2_treino': np.mean(scores_unscaled['train_r2']), 'R2_teste': np.mean(scores_unscaled['test_r2']),
                'RMSE_treino': -np.mean(scores_unscaled['train_rmse']), 'RMSE_teste': -np.mean(scores_unscaled['test_rmse'])
            })

            # Cenário ESCALONADO
            print(f"    - Avaliando: {model_name}_Scaled")
            pipeline_scaled = Pipeline([('scaler', StandardScaler()), ('model', model_instance)])
            scores_scaled = cross_validate(pipeline_scaled, X_subset, y, cv=cv_splitter, scoring=scoring, groups=groups, return_train_score=True)
            resultados_finais.append({
                'experimento': f"{model_name}_Scaled ({feature_set_name})",
                'R2_treino': np.mean(scores_scaled['train_r2']), 'R2_teste': np.mean(scores_scaled['test_r2']),
                'RMSE_treino': -np.mean(scores_scaled['train_rmse']), 'RMSE_teste': -np.mean(scores_scaled['test_rmse'])
            })



--- Iniciando a avaliação dos modelos com hiperparâmetros otimizados ---

>>> Avaliando com o conjunto de features: 'simple'
    - Avaliando: Lasso_Scaled
    - Avaliando: DecisionTree_Unscaled
    - Avaliando: DecisionTree_Scaled
    - Avaliando: RandomForest_Unscaled
    - Avaliando: RandomForest_Scaled
    - Avaliando: XGBoost_Unscaled
    - Avaliando: XGBoost_Scaled

>>> Avaliando com o conjunto de features: 'simple_com_velocidade'
    - Avaliando: Lasso_Scaled
    - Avaliando: DecisionTree_Unscaled
    - Avaliando: DecisionTree_Scaled
    - Avaliando: RandomForest_Unscaled
    - Avaliando: RandomForest_Scaled
    - Avaliando: XGBoost_Unscaled
    - Avaliando: XGBoost_Scaled

>>> Avaliando com o conjunto de features: 'ambiental_completo'
    - Avaliando: Lasso_Scaled
    - Avaliando: DecisionTree_Unscaled
    - Avaliando: DecisionTree_Scaled
    - Avaliando: RandomForest_Unscaled
    - Avaliando: RandomForest_Scaled
    - Avaliando: XGBoost_Unscaled
    - Avaliando: XGBoost_Scaled

In [9]:
# =============================================================================
# 5. APRESENTAÇÃO DOS RESULTADOS
# =============================================================================
results_df = pd.DataFrame(resultados_finais)
print("\n--- Avaliação concluída ---")
print("Tabela de Resultados Comparativos (ordenada pelo melhor RMSE de teste):")
pd.set_option('display.max_colwidth', 100)
display(results_df.sort_values('RMSE_teste'))



--- Avaliação concluída ---
Tabela de Resultados Comparativos (ordenada pelo melhor RMSE de teste):


,experimento,R2_treino,R2_teste,RMSE_treino,RMSE_teste
7,Lasso_Scaled (simple_com_velocidade),0.820005,0.806649,0.405351,0.419143
10,RandomForest_Unscaled (simple_com_velocidade),0.853515,0.805413,0.365684,0.420605
20,XGBoost_Scaled (ambiental_completo),0.857647,0.805199,0.360454,0.420635
19,XGBoost_Unscaled (ambiental_completo),0.857647,0.805199,0.360454,0.420635
11,RandomForest_Scaled (simple_com_velocidade),0.853529,0.805346,0.365665,0.420679
0,Lasso_Scaled (simple),0.810094,0.804452,0.416336,0.421392
14,Lasso_Scaled (ambiental_completo),0.812921,0.802784,0.413208,0.423018
18,RandomForest_Scaled (ambiental_completo),0.851959,0.802483,0.367596,0.423469
17,RandomForest_Unscaled (ambiental_completo),0.851958,0.802453,0.367598,0.423494
5,XGBoost_Unscaled (simple),0.860523,0.801959,0.356785,0.424308
